First install dependencies:

pip install openmeteo-requests
pip install requests-cache retry-requests numpy pandas

In [7]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

In [9]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
responses = []
df = pd.DataFrame()

In [23]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
	"latitude": 51.5085,
	"longitude": -0.1257,
	"start_date": "2015-01-01",
	"end_date": "2020-01-01",
	"daily": ["weather_code",
           "temperature_2m_mean",
           "temperature_2m_max",
           "temperature_2m_min",
           "precipitation_sum",
           "rain_sum",
           "snowfall_sum",
           "precipitation_hours",
           "wind_speed_10m_max",
           "wind_gusts_10m_max",
           "wind_direction_10m_dominant",
           "shortwave_radiation_sum"]
}

#coordinates = {'madrid':{'latitud':'40,4165', 'longitude':'-3,7026'},
#               'london':{'latitud':'51,5085', 'longitude':'-0,1257'},
#               'paris':{'latitud':'48,8534', 'longitude':'2,3488'},
#               'berlin':{'latitud':'52,5244', 'longitude':'13,4105'},
#               'brussels':{'latitud':'50,8505', 'longitude':'4,3488'},
#               'istanbul':{'latitud':'41,0138', 'longitude':'28,9497'}
#}

In [24]:
def get_weather_data():
    response = openmeteo.weather_api(url, params=params)
    return response

In [25]:
def get_weather_dataframe():
    daily_dataframe = pd.DataFrame()
    response = get_weather_data()
    daily = response[0].Daily()
    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_max = daily.Variables(2).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(3).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(4).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(5).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(6).ValuesAsNumpy()
    daily_precipitation_hours = daily.Variables(7).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(8).ValuesAsNumpy()
    daily_wind_gusts_10m_max = daily.Variables(9).ValuesAsNumpy()
    daily_wind_direction_10m_dominant = daily.Variables(10).ValuesAsNumpy()
    daily_shortwave_radiation_sum = daily.Variables(11).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["weather_code"] = daily_weather_code
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum
    daily_data["precipitation_hours"] = daily_precipitation_hours
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
    daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
    daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum

    temp_df = pd.DataFrame(data = daily_data)
    daily_dataframe = pd.concat([daily_dataframe,temp_df])
    return daily_dataframe


In [26]:
df = get_weather_dataframe()

In [28]:
display(df)

,date,weather_code,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum
0,2015-01-01 00:00:00+00:00,53.0,7.818750,11.150000,4.7000,1.4,1.4,0.0,3.0,34.296494,70.919998,210.161819,0.90
1,2015-01-02 00:00:00+00:00,51.0,7.533333,12.400000,3.0000,0.2,0.2,0.0,1.0,33.371437,64.439995,256.971161,3.88
2,2015-01-03 00:00:00+00:00,61.0,3.958333,5.050000,2.8000,11.4,11.4,0.0,13.0,18.356470,51.119999,270.000061,0.43
3,2015-01-04 00:00:00+00:00,3.0,2.195833,3.700000,-0.3000,0.0,0.0,0.0,0.0,9.885262,19.799999,244.129440,3.34
4,2015-01-05 00:00:00+00:00,3.0,6.466666,9.350000,3.8500,0.0,0.0,0.0,0.0,14.255272,31.319998,200.189224,1.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,2019-12-28 00:00:00+00:00,3.0,7.802333,9.656500,6.3065,0.0,0.0,0.0,0.0,13.708391,23.759998,184.041275,1.77
1823,2019-12-29 00:00:00+00:00,3.0,5.929417,8.856501,3.4065,0.0,0.0,0.0,0.0,15.937878,27.359999,203.607742,3.47
1824,2019-12-30 00:00:00+00:00,3.0,5.448166,10.056500,2.2065,0.0,0.0,0.0,0.0,16.575644,28.440001,234.087982,3.77
1825,2019-12-31 00:00:00+00:00,3.0,5.952333,9.256500,2.2065,0.0,0.0,0.0,0.0,16.981165,28.440001,84.347328,1.49


In [20]:
df.to_csv('weather_output.csv')